In [ ]:
#Sandbox for playing around with heading-based segmentation, etc.

In [5]:
import movingpandas as mpd
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.geometry import Point
import pandas as pd
import numpy as np
import hvplot.pandas
import geojson

In [6]:
file = 'ooinhdeep_points.geojson'

In [7]:
with open(file) as f:
    gj = geojson.load(f)
features = gj['features'][0]

In [10]:
#Extract lats and lons
lats = [feature['properties']['latitude'] for feature in gj['features']]
lons = [feature['properties']['longitude'] for feature in gj['features']]

In [13]:
df = pd.DataFrame({'lat': lats, 'lon': lons})
df.hvplot(x='lon', y='lat')

:Curve   [lon]   (lat)

In [14]:
#Create a GeoDataFrame
geom = [Point(x,y) for x, y in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry = geom)
gdf = gdf.set_crs(epsg=4326)

In [16]:
#Get gdf with differences between x and y for calculating angle with arctan. Think about what to do with last point
gdf_diff = gdf.diff(periods=-1)

/Users/katyabbott/miniconda3/envs/oceanpy/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: dtype lost in 'diff()'. In the future this will raise a TypeError. Convert to a suitable dtype prior to calling 'diff'.
  


In [24]:
gdf['theta'] = np.arctan2(gdf_diff['lat'], gdf_diff['lon'])

In [264]:
tol = .3
n = 500

In [30]:
def iterative_segmentation(gdf, criterion, tol):
    """
    gdf is the GeoDataFrame with tracks that you're interested in
    criterion is a column name in the gdf that you're using to segment — for example, angle if you're looking
        at changes in heading, or speed
    tol is the tolerance to a given criterion. For example, a tolerance of .3 means that a change of .3
        in criterion value between two rows will be allowed
    """
    trajectory_id = []
    j = 0 #i.e., the first trajectory
    for i, row in gdf.iterrows():
        #Assign starting value for the criterion
        if i == 0:
            start_value = row[criterion]
        else:
            if abs(row[criterion] - start_value) < tol:
                pass
            elif abs(row[criterion] - start_value) >= tol:
                j+=1 #Start a new trajectory label
        start_value = row[criterion] #Update comparison value. This method is more robust to changes in curvature
        #that may not represent a new trajectory
        trajectory_id.append(j)
    gdf['traj_id'] = trajectory_id
    return gdf

In [31]:
def direction_segmentation(gdf, criterion, tol):
    """
    gdf is the GeoDataFrame with tracks that you're interested in
    criterion is a column name in the gdf that you're using to segment — for example, angle if you're looking
        at changes in heading, or speed
    tol is the tolerance to a given criterion. For example, a tolerance of .3 means that a change of .3
        in criterion value between two rows will be allowed
    """
    trajectory_label = []
    j = 0 #i.e., the first trajectory
    for i, row in gdf.iterrows():
        #Assign starting value for the criterion
        if i == 0:
            start_value = row[criterion]
        else:
            if abs(row[criterion] - start_value) < tol:
                pass
            elif abs(row[criterion] - start_value) >= tol:
                start_angle = row[criterion] #only change starting angle if we're starting a new trajectory
                j+=1 #Start a new trajectory label
         #Update 
        trajectory_label.append(j)
    gdf['traj_id'] = trajectory_id
    return gdf

In [54]:
#Apply the iterative segmentation approach on angle with a tolerance of 0.3 
seg_gdf = iterative_segmentation(gdf, 'theta', .3)

In [67]:
traj_collection = mpd.TrajectoryCollection(seg_gdf, 'traj_id')

In [51]:
#For comparison with original gdf
gdf['traj_id'] = 1
traj_collection1 = mpd.TrajectoryCollection(gdf, 'traj_id')

In [68]:
plot = traj_collection.hvplot()
hvplot.save(plot, 'gliders.html')

In [ ]:
## Comparison with original tracks

In [62]:
traj_collection.trajectories[0].df.drop('geometry', axis = 1).hvplot.line(x='lon', y='lat') * traj_collection.trajectories[0].df.drop('geometry', axis = 1).hvplot.scatter(x='lon', y='lat')

:Overlay
   .Curve.I   :Curve   [lon]   (lat)
   .Scatter.I :Scatter   [lon]   (lat)

In [64]:
traj_collection1.trajectories[0].df.drop('geometry', axis = 1).head(200).hvplot(x='lon', y='lat')

:Curve   [lon]   (lat)